In [3]:
from cloud_dataset import CloudDataset
from losses import intersection_over_union
import pandas as pd
import matplotlib.pyplot as plt
import xrspatial.multispectral as ms
import torch
import rioxarray
from torch.utils.data import DataLoader
import torchvision.transforms.functional as TF
import torch.nn as nn
import numpy as np
import torch.optim as optim
import multiprocessing
import warnings
warnings.filterwarnings("ignore")
import torchvision.transforms as transforms
from PIL import Image as im
from tqdm.notebook import tqdm_notebook
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import segmentation_models_pytorch as smp

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
BANDS = ["B02", "B03", "B04", "B08"]

In [6]:
train_x = pd.read_csv("data/train_x.csv").drop(columns = "Unnamed: 0")
train_y = pd.read_csv("data/train_y.csv").drop(columns = "Unnamed: 0")  
val_x = pd.read_csv("data/val_x.csv").drop(columns = "Unnamed: 0")    
val_y = pd.read_csv("data/val_y.csv").drop(columns = "Unnamed: 0")

In [7]:
train_x

,chip_id,B02_path,B03_path,B04_path,B08_path
0,adwp,/driven-data/cloud-cover/train_features/adwp/B...,/driven-data/cloud-cover/train_features/adwp/B...,/driven-data/cloud-cover/train_features/adwp/B...,/driven-data/cloud-cover/train_features/adwp/B...
1,adwu,/driven-data/cloud-cover/train_features/adwu/B...,/driven-data/cloud-cover/train_features/adwu/B...,/driven-data/cloud-cover/train_features/adwu/B...,/driven-data/cloud-cover/train_features/adwu/B...
2,adxp,/driven-data/cloud-cover/train_features/adxp/B...,/driven-data/cloud-cover/train_features/adxp/B...,/driven-data/cloud-cover/train_features/adxp/B...,/driven-data/cloud-cover/train_features/adxp/B...
3,aeaj,/driven-data/cloud-cover/train_features/aeaj/B...,/driven-data/cloud-cover/train_features/aeaj/B...,/driven-data/cloud-cover/train_features/aeaj/B...,/driven-data/cloud-cover/train_features/aeaj/B...
4,aeap,/driven-data/cloud-cover/train_features/aeap/B...,/driven-data/cloud-cover/train_features/aeap/B...,/driven-data/cloud-cover/train_features/aeap/B...,/driven-data/cloud-cover/train_features/aeap/B...
...,...,...,...,...,...
7866,zxpu,/driven-data/cloud-cover/train_features/zxpu/B...,/driven-data/cloud-cover/train_features/zxpu/B...,/driven-data/cloud-cover/train_features/zxpu/B...,/driven-data/cloud-cover/train_features/zxpu/B...
7867,zxwv,/driven-data/cloud-cover/train_features/zxwv/B...,/driven-data/cloud-cover/train_features/zxwv/B...,/driven-data/cloud-cover/train_features/zxwv/B...,/driven-data/cloud-cover/train_features/zxwv/B...
7868,zxxo,/driven-data/cloud-cover/train_features/zxxo/B...,/driven-data/cloud-cover/train_features/zxxo/B...,/driven-data/cloud-cover/train_features/zxxo/B...,/driven-data/cloud-cover/train_features/zxxo/B...
7869,zxym,/driven-data/cloud-cover/train_features/zxym/B...,/driven-data/cloud-cover/train_features/zxym/B...,/driven-data/cloud-cover/train_features/zxym/B...,/driven-data/cloud-cover/train_features/zxym/B...


In [5]:
all_x = pd.concat([train_x, val_x], axis = 0).reset_index()
all_y = pd.concat([train_y, val_y], axis = 0).reset_index()

In [6]:
all_y

,index,chip_id,label_path
0,0,adwp,/driven-data/cloud-cover/train_labels/adwp.tif
1,1,adwu,/driven-data/cloud-cover/train_labels/adwu.tif
2,2,adxp,/driven-data/cloud-cover/train_labels/adxp.tif
3,3,aeaj,/driven-data/cloud-cover/train_labels/aeaj.tif
4,4,aeap,/driven-data/cloud-cover/train_labels/aeap.tif
...,...,...,...
11743,3872,zxrt,/driven-data/cloud-cover/train_labels/zxrt.tif
11744,3873,zxsj,/driven-data/cloud-cover/train_labels/zxsj.tif
11745,3874,zxuw,/driven-data/cloud-cover/train_labels/zxuw.tif
11746,3875,zxvi,/driven-data/cloud-cover/train_labels/zxvi.tif


In [7]:
train_transforms = A.Compose(
        [
            # A.Resize(height=256, width=256),
            A.Rotate(limit=60, p=0.6),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            ToTensorV2(),
        ],
    )

val_transforms = A.Compose(
    [
        # A.Resize(height=256, width=256),
        ToTensorV2(),
    ]
)

In [8]:
trainset = CloudDataset(train_x, BANDS, train_y, train_transforms, norm = True)
valset = CloudDataset(val_x, BANDS, val_y, val_transforms, norm = True)
allset = CloudDataset(all_x, BANDS, all_y, train_transforms, norm = True)

In [9]:
train_loader = DataLoader(dataset = trainset, batch_size = 16, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(dataset = valset, batch_size = 16, shuffle = False, pin_memory=True, num_workers=4)
all_loader  = DataLoader(dataset = allset, batch_size = 16, shuffle = True, pin_memory=True, num_workers=4)

In [11]:
model = smp.DeepLabV3Plus(
    encoder_name="resnext101_32x4d",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="ssl",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=4,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=2,                      # model output channels (number of classes in your dataset)
).to(device)

In [13]:
all_loader = [train_loader, val_loader]

In [12]:
!pip install torchinfo

In [13]:
# from torchinfo import summary
# summary(model, input_size=(16, 4, 512, 512))

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.00005)
scaler = torch.cuda.amp.GradScaler()

In [15]:
model.load_state_dict(torch.load("AGall0deep_plus.pt"))

<All keys matched successfully>

In [16]:
def train_fn(loader, model, optimizer,loss_fn, scaler):
    loadtq = tqdm_notebook(loader, desc = "batches")
    for it in loadtq:
        data = it["chip"].type(torch.FloatTensor)
        targets = it["label"]
        data = data.to(device)
        targets = targets.type(torch.LongTensor).to(device)
        with torch.cuda.amp.autocast():
            predictions = model(data)
            # print(predictions.shape)
            # print(predictions["out"].shape)
            loss = loss_fn(predictions, targets)
            loss = loss.to(device)
            
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        loadtq.set_postfix(loss=loss.item())

In [1]:
for i in range(25):
    train_fn(all_loader, model, optimizer, loss_fn, scaler)
    torch.save(model.state_dict(), f"OOAGall{i}deep_plus.pt" )

In [ ]:
model.eval()
ioulis = []
loadtq  = tqdm_notebook(val_loader)

with torch.no_grad():
    for it in loadtq:
        inpic = it["chip"].to(device).type(torch.cuda.FloatTensor)
        true = it["label"].squeeze()
        pred = model(inpic)
        predic = torch.argmax(pred, dim = 1).squeeze()
        iou = intersection_over_union(predic.detach().to("cpu"), true.detach().to("cpu"))
        ioulis.append(iou)
        loadtq.set_postfix(iou = iou)
    
model.train()

In [ ]:
ioulis = np.array(ioulis)

In [ ]:
ioulis.mean()